In [135]:
from pathlib import Path
from ast import literal_eval
from collections import Counter
import pickle
from typing import Tuple

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import tqdm

In [96]:
thresholds

array([0.        , 0.        , 0.        , ..., 0.04525986, 0.04580943,
       0.04611834], dtype=float32)

In [108]:
final =[]
for i in range (0, 5):
    for set_ in ["train"]:
        X = pickle.load(open(f"C:/Users/benna/Documents/adn_circulant/data/for_training/X_{set_}_{i}.pkl", 'rb'))
        y = pickle.load(open(f"C:/Users/benna/Documents/adn_circulant/data/for_training/y_{set_}_{i}.pkl",'rb'))
        model = XGBClassifier(tree_method="hist")
        model.fit(X, y)
        # extract the feature importances
        thresholds = np.sort(model.feature_importances_)
        first_not_nul = [i for i in thresholds if i != 0][0]
        # select features using threshold
        selection = SelectFromModel(model,threshold=first_not_nul, prefit=True)
        feature_idx = selection.get_support()
        feature_name = X.columns[feature_idx]
        final =list(set(feature_name) | set(final))

In [139]:
len(final)

520

# Mmodèle après feature selection

In [137]:
def cross_val(xgb):
    acc_cv = []
    for i in range (0,5):
        X_test = pickle.load(open(f"C:/Users/benna/Documents/adn_circulant/data/for_training/X_test_{i}.pkl", 'rb'))
        y_test = pickle.load(open(f"C:/Users/benna/Documents/adn_circulant/data/for_training/y_test_{i}.pkl",'rb'))
        X_train = pickle.load(open(f"C:/Users/benna/Documents/adn_circulant/data/for_training/X_train_{i}.pkl", 'rb'))
        y_train = pickle.load(open(f"C:/Users/benna/Documents/adn_circulant/data/for_training/y_train_{i}.pkl",'rb'))
        #---------- feature selection 
        model = XGBClassifier(tree_method="hist")
        model.fit(X, y)
        # extract the feature importances
        thresholds = np.sort(model.feature_importances_)
        first_not_nul = [i for i in thresholds if i != 0][0]
        # select features using threshold
        selection = SelectFromModel(model,threshold=first_not_nul, prefit=True)
        feature_idx = selection.get_support()
        feature_name = X.columns[feature_idx]
        print (f"selected feature: {len(feature_name)}")
        X_train_selected = X_train[feature_name]
        X_test_selected = X_test[feature_name]
        xgb.fit(X_train_selected, y_train)
            
        y_pred = xgb.predict(X_train_selected)
        print(f'Accuracy on train : {round(100 * accuracy_score(y_pred, y_train))} % for set {i}')
            
        y_pred = xgb.predict(X_test_selected)
        print(f'Accuracy on test : {round(100 * accuracy_score(y_pred, y_test))} % for set {i}')
        acc_cv.append(accuracy_score(y_pred, y_test))
    return np.mean(acc_cv), acc_cv

In [138]:
xgb = XGBClassifier(tree_method = "hist")
cross_val(xgb)

selected feature: 139
Accuracy on train : 100.0 % for set 0
Accuracy on test : 91.0 % for set 0
selected feature: 139
Accuracy on train : 100.0 % for set 1
Accuracy on test : 79.0 % for set 1
selected feature: 139
Accuracy on train : 100.0 % for set 2
Accuracy on test : 71.0 % for set 2
selected feature: 139
Accuracy on train : 100.0 % for set 3
Accuracy on test : 68.0 % for set 3
selected feature: 139
Accuracy on train : 100.0 % for set 4
Accuracy on test : 67.0 % for set 4


(0.7509803921568626,
 [0.9117647058823529,
  0.7941176470588235,
  0.7058823529411765,
  0.6764705882352942,
  0.6666666666666666])

In [140]:
rf = RandomForestClassifier(n_estimators=1000)
cross_val(rf)

selected feature: 139
Accuracy on train : 100.0 % for set 0
Accuracy on test : 82.0 % for set 0
selected feature: 139
Accuracy on train : 100.0 % for set 1
Accuracy on test : 76.0 % for set 1
selected feature: 139
Accuracy on train : 100.0 % for set 2
Accuracy on test : 79.0 % for set 2
selected feature: 139
Accuracy on train : 100.0 % for set 3
Accuracy on test : 68.0 % for set 3
selected feature: 139
Accuracy on train : 100.0 % for set 4
Accuracy on test : 76.0 % for set 4


(0.7632798573975045,
 [0.8235294117647058,
  0.7647058823529411,
  0.7941176470588235,
  0.6764705882352942,
  0.7575757575757576])

In [141]:
lr = LogisticRegression(class_weight='balanced', C=2)
cross_val(lr)

selected feature: 139
Accuracy on train : 92.0 % for set 0
Accuracy on test : 76.0 % for set 0
selected feature: 139
Accuracy on train : 88.0 % for set 1
Accuracy on test : 79.0 % for set 1
selected feature: 139
Accuracy on train : 87.0 % for set 2
Accuracy on test : 82.0 % for set 2
selected feature: 139
Accuracy on train : 87.0 % for set 3
Accuracy on test : 76.0 % for set 3
selected feature: 139
Accuracy on train : 92.0 % for set 4
Accuracy on test : 64.0 % for set 4


(0.7566844919786095,
 [0.7647058823529411,
  0.7941176470588235,
  0.8235294117647058,
  0.7647058823529411,
  0.6363636363636364])